К сожалению методами ИИ задачу решить пока не получилось. Перевод сделан простыми алгоритмами. Пока не все сделано, программа будет совершенствоваться.

In [63]:
import pandas as pd

import requests
import re
import string

import pymorphy2
lemmer = pymorphy2.MorphAnalyzer()

In [15]:
df = pd.read_csv('../lr3/witcher_words.csv')

In [74]:
def translate_russian_to_elvish(text: str) -> str:
    """Переводит русский текст на язык Старшей Речи."""
    # Унифицируем е/ё:
    text = text.replace('ё', 'е')

    # Бьём текст на части по пробелам.
    translated_parts = []

    parts = text.split()
    for part in parts:

        # Проверяем не является ли текущая часть набором знаков.
        is_punctuation = True
        for s in part:
            if s not in string.punctuation:
                is_punctuation = False
                break

        if is_punctuation:
            translated_parts.append(part)
            continue

        part_len = len(part)

        # Собираем знаки слева.
        start_part_i = 0
        for i in range(part_len):
            if part[i] in string.punctuation:
                start_part_i += 1
            else:
                break

        # Собираем знаки справа.
        end_part_i = part_len
        for i in range(part_len):
            if part[part_len - i - 1] in string.punctuation:
                end_part_i -= 1
            else:
                break

        # Распределяем на три части: знаки слева, чистое слово, знаки справа.
        word = part[start_part_i:end_part_i]
        start_part = part[0:start_part_i]
        end_part = part[end_part_i:]

        # Флаг для сохранения регистра первой буквы.
        is_title = word.replace('\'', '').istitle()

        # Подбираем перевод слова.
        word = word.lower()
        word = _word_to_lemm(word)
        translation = None
        try:
            translation = _try_to_find_russian_word_translation(word)
        except ValueError:
            for synonym in _synonyms_of_word(word):
                try:
                    translation = _try_to_find_russian_word_translation(synonym)
                    break
                except ValueError:
                    continue

        # Если перевода не нашлось, то делаем ужасные вещи...
        if translation is None:
            translation = transliterate_russian_to_latin(word)

        # Возвращаем регистр первой буквы.
        if is_title:
            translation = translation.title()

        # Формируем полную часть.
        translation_part = start_part + translation + end_part
        translated_parts.append(translation_part)

    return ' '.join(translated_parts)


def _word_to_lemm(word: str) -> str:
    """Переводит слово/словосочетание в начальную форму."""
    parts = []
    for part in word.split():
        part = lemmer.parse(part)[0].normal_form
        parts.append(part)

    return ' '.join(parts)


def _try_to_find_russian_word_translation(word: str) -> str:
    """Ищет перевод русского слова в датасете."""
    result = df.loc[df['translation'] == word, 'text']
    if not result.empty:
        # phrases = df.loc[df['translation'].str.startswith(word) & df['translation'].str.contains(' '), 'translation'].tolist()
        return result.values[0]
    else:
        raise ValueError


def _synonyms_of_word(word: str) -> str:
    """Выдаёт всевозможные синонимы заданного слова. Использует сетевой запрос."""
    html = requests.get(f'https://text.ru/synonym/{word}').text
    
    match = re.search(r'<meta name=\"description\" content=\"Синонимы к слову [^—]*:([^\"]+)\" />', html)
    if match is None:
        return []

    text = match.group(1)
    synonyms = [synonym.strip() for synonym in text.strip().split('—')]
    synonyms.remove('')

    return synonyms


def transliterate_russian_to_latin(russian_word: str) -> str:
    """Функция для латинизации русского слова."""
    translit_dict = {
        'а': 'a', 
        'б': 'b', 
        'в': 'v', 
        'г': 'g', 
        'д': 'd', 
        'е': 'e', 
        'ё': 'yo',
        'ж': 'zh', 
        'з': 'z', 
        'и': 'i', 
        'й': 'y', 
        'к': 'k', 
        'л': 'l', 
        'м': 'm',
        'н': 'n', 
        'о': 'o', 
        'п': 'p', 
        'р': 'r', 
        'с': 's', 
        'т': 't', 
        'у': 'u',
        'ф': 'f', 
        'х': 'kh', 
        'ц': 'ts', 
        'ч': 'ch', 
        'ш': 'sh', 
        'щ': 'shch', 
        'ъ': '',
        'ы': 'y', 
        'ь': '', 
        'э': 'e', 
        'ю': 'yu', 
        'я': 'ya',
    }

    latin_word = ''.join(translit_dict.get(symbol, symbol) for symbol in russian_word.lower())
    return latin_word

In [75]:
translate_russian_to_elvish('Здравствуй, Геральт Белый Волк! Моё имя - Аваллак\'х! Помоги мне найти ласточку!')

"Cead, Geralt Gwen Bleidd! Me aymm - Avallak'Kha! Pomoch aé nayti zireael!"

In [76]:
translate_russian_to_elvish('Прощай, дочь. Мы еще встретимся.')

'Va Faill, luned. Sinn aris vstretitsya.'

In [77]:
translate_russian_to_elvish('Когда обед? Я хочу есть.')

'Cathain obed? Aé wett ithte.'

In [78]:
translate_russian_to_elvish('Почему этот курс такой сложный? Я не знаю.')

'Hinc etot rider takoy slozhnyy? Aé neen ifit.'

In [79]:
translate_russian_to_elvish('Замок Моря прекрасен летом. Он здесь как картина.')

'Caer Muire elaine eate. Ei anseo conas eie.'